In [22]:
from metatrader.utils.factory import MetatraderFactory
metatrader = MetatraderFactory.get_metatrader()
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
import pytz

mt5 = metatrader.connect()

symbol = 'AUDUSD'
timeframe = mt5.TIMEFRAME_D1
utc_from = pytz.timezone("UTC").localize(datetime(2023, 10, 28))

bars = pd.DataFrame(mt5.copy_rates_from(symbol, timeframe, utc_from, 100000))
bars['time'] = pd.to_datetime(bars['time'], unit='s')



Connected to Metatrader successfully.


In [23]:
bars

,time,open,high,low,close,tick_volume,spread,real_volume
0,2000-01-03,0.65630,0.66150,0.65570,0.65840,1988,50,0
1,2000-01-04,0.65830,0.65940,0.65410,0.65480,2919,50,0
2,2000-01-05,0.65460,0.66000,0.65350,0.65830,3213,50,0
3,2000-01-06,0.65810,0.65890,0.65250,0.65290,3457,50,0
4,2000-01-07,0.65280,0.65620,0.65220,0.65550,2462,50,0
...,...,...,...,...,...,...,...,...
6273,2023-10-23,0.63189,0.63485,0.62886,0.63359,39714,3,0
6274,2023-10-24,0.63362,0.63788,0.63351,0.63598,41457,3,0
6275,2023-10-25,0.63558,0.63994,0.62845,0.62937,43502,3,0
6276,2023-10-26,0.62928,0.63346,0.62699,0.63330,45251,3,0


In [24]:
def get_daily_bias(x):
    if x["close"] < x["open"]:
        return -1
    elif x["close"] > x["open"]:
        return 1

In [25]:
bars['daily_bias'] = bars.apply(get_daily_bias, axis=1)

bars.fillna(1, inplace=True)

bars

,time,open,high,low,close,tick_volume,spread,real_volume,daily_bias
0,2000-01-03,0.65630,0.66150,0.65570,0.65840,1988,50,0,1.0
1,2000-01-04,0.65830,0.65940,0.65410,0.65480,2919,50,0,-1.0
2,2000-01-05,0.65460,0.66000,0.65350,0.65830,3213,50,0,1.0
3,2000-01-06,0.65810,0.65890,0.65250,0.65290,3457,50,0,-1.0
4,2000-01-07,0.65280,0.65620,0.65220,0.65550,2462,50,0,1.0
...,...,...,...,...,...,...,...,...,...
6273,2023-10-23,0.63189,0.63485,0.62886,0.63359,39714,3,0,1.0
6274,2023-10-24,0.63362,0.63788,0.63351,0.63598,41457,3,0,1.0
6275,2023-10-25,0.63558,0.63994,0.62845,0.62937,43502,3,0,-1.0
6276,2023-10-26,0.62928,0.63346,0.62699,0.63330,45251,3,0,1.0


In [26]:
def calculate_high_low_of_day(row):
    day_start_date_object = datetime.strptime(str(row["time"]), "%Y-%m-%d %H:%M:%S")
    day_start_date_object = day_start_date_object + timedelta(days=1)
    daily_df = pd.DataFrame(mt5.copy_rates_from(
        symbol,
        mt5.TIMEFRAME_H1,
        day_start_date_object,
        23
    ))
    daily_df['time'] = pd.to_datetime(daily_df['time'], unit='s')

    daily_df.fillna(1, inplace=True)

    if row["daily_bias"] == 1:
        hour_of_the_day = [
            daily_df[daily_df["low"] == row["low"]]["time"]
            .dt.strftime("%H")
            .values
        ][0].tolist()
        if len(hour_of_the_day):
            return hour_of_the_day[0]
        else:
            return "00"
    elif row["daily_bias"] == -1:
        hour_of_the_day = [
            daily_df[daily_df["high"] == row["high"]]["time"]
            .dt.strftime("%H")
            .values
        ][0].tolist()
        if len(hour_of_the_day):
            return hour_of_the_day[0]
        else:
            return "00"

In [27]:
bars["day_high_low_hour"] = bars.apply(
    calculate_high_low_of_day, axis=1
)



In [ ]:
bars.to_csv(f'./csv/daily_{symbol}.csv')